In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from datetime import datetime
import json
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from google.colab import files
import zipfile
import gc

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Enable memory growth to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

class UNetOilSpillSegmenter:
    def __init__(self, input_shape=(256, 256, 3)):
        self.input_shape = input_shape
        self.model = None
        self.history = None
        self.metrics = {}
        self.class_weights = None

    def build_unet(self, filters=32, dropout_rate=0.3, batch_norm=True):  # Reduced filters
        """
        Build U-Net model for oil spill segmentation with customizable parameters
        """
        inputs = Input(self.input_shape)

        # Encoder (Contracting Path)
        def conv_block(x, filters, dropout_rate=0.0, batch_norm=True):
            x = Conv2D(filters, 3, padding='same')(x)
            if batch_norm:
                x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(filters, 3, padding='same')(x)
            if batch_norm:
                x = BatchNormalization()(x)
            x = Activation('relu')(x)

            if dropout_rate > 0:
                x = Dropout(dropout_rate)(x)
            return x

        # Encoder
        c1 = conv_block(inputs, filters, 0.0, batch_norm)
        p1 = MaxPooling2D((2, 2))(c1)

        c2 = conv_block(p1, filters*2, 0.0, batch_norm)
        p2 = MaxPooling2D((2, 2))(c2)

        c3 = conv_block(p2, filters*4, 0.0, batch_norm)
        p3 = MaxPooling2D((2, 2))(c3)

        c4 = conv_block(p3, filters*8, dropout_rate, batch_norm)
        p4 = MaxPooling2D((2, 2))(c4)

        # Bottleneck
        c5 = conv_block(p4, filters*16, dropout_rate, batch_norm)

        # Decoder (Expansive Path)
        u6 = UpSampling2D((2, 2))(c5)
        u6 = Conv2D(filters*8, 2, padding='same')(u6)
        u6 = concatenate([u6, c4])
        c6 = conv_block(u6, filters*8, 0.0, batch_norm)

        u7 = UpSampling2D((2, 2))(c6)
        u7 = Conv2D(filters*4, 2, padding='same')(u7)
        u7 = concatenate([u7, c3])
        c7 = conv_block(u7, filters*4, 0.0, batch_norm)

        u8 = UpSampling2D((2, 2))(c7)
        u8 = Conv2D(filters*2, 2, padding='same')(u8)
        u8 = concatenate([u8, c2])
        c8 = conv_block(u8, filters*2, 0.0, batch_norm)

        u9 = UpSampling2D((2, 2))(c8)
        u9 = Conv2D(filters, 2, padding='same')(u9)
        u9 = concatenate([u9, c1])
        c9 = conv_block(u9, filters, 0.0, batch_norm)

        # Output layer
        outputs = Conv2D(1, 1, activation='sigmoid')(c9)

        # Create model
        model = Model(inputs=inputs, outputs=outputs)
        self.model = model
        return model

    def dice_coefficient(self, y_true, y_pred, smooth=1e-6):
        """
        Dice coefficient metric for segmentation evaluation
        """
        # Flatten both tensors to ensure consistent shapes
        y_true_f = tf.reshape(y_true, [-1])
        y_pred_f = tf.reshape(y_pred, [-1])

        intersection = tf.reduce_sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

    def dice_loss(self, y_true, y_pred):
        """
        Dice loss for segmentation
        """
        return 1 - self.dice_coefficient(y_true, y_pred)

    def bce_dice_loss(self, y_true, y_pred):
        """
        Combined BCE and Dice loss
        """
        # Flatten both tensors to ensure consistent shapes
        y_true_flat = tf.reshape(y_true, [-1])
        y_pred_flat = tf.reshape(y_pred, [-1])

        bce = tf.keras.losses.binary_crossentropy(y_true_flat, y_pred_flat)
        dice = self.dice_loss(y_true, y_pred)
        return bce + dice

    def compile_model(self, learning_rate=1e-4, loss_function='bce_dice'):
        """
        Compile the model with specified loss function
        """
        if self.model is None:
            raise ValueError("Model not built yet. Please build the model first.")

        # Select loss function
        if loss_function == 'bce_dice':
            loss = self.bce_dice_loss
        elif loss_function == 'dice':
            loss = self.dice_loss
        else:
            loss = 'binary_crossentropy'

        # Compile model with fewer metrics to save memory
        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss=loss,
            metrics=['accuracy', self.dice_coefficient]  # Reduced metrics
        )

    def train(self, X_train, y_train, X_val, y_val, epochs=20, batch_size=8):  # Reduced batch size
        """
        Train the U-Net model
        """
        if self.model is None:
            raise ValueError("Model not built yet. Please build the model first.")

        # Memory-efficient callbacks
        callbacks = [
            ModelCheckpoint(
                '/content/best_unet_model.h5',
                monitor='val_loss',
                save_best_only=True,
                mode='min',
                verbose=1
            ),
            EarlyStopping(
                monitor='val_loss',
                patience=10,  # Reduced patience
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,
                patience=5,  # Reduced patience
                min_lr=1e-7,
                verbose=1
            )
        ]

        # Train the model with smaller batch size
        self.history = self.model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )

        return self.history

    def evaluate(self, X_test, y_test, batch_size=8):
        """
        Evaluate the model on test data in batches to save memory
        """
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        # Evaluate in batches
        num_batches = int(np.ceil(len(X_test) / batch_size))
        metrics_accum = {
            'loss': 0,
            'accuracy': 0,
            'dice_coefficient': 0
        }

        for i in range(num_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, len(X_test))

            X_batch = X_test[start_idx:end_idx]
            y_batch = y_test[start_idx:end_idx]

            results = self.model.evaluate(X_batch, y_batch, verbose=0)
            metrics_accum['loss'] += results[0] * len(X_batch)
            metrics_accum['accuracy'] += results[1] * len(X_batch)
            metrics_accum['dice_coefficient'] += results[2] * len(X_batch)

        # Average metrics
        metrics = {
            'loss': float(metrics_accum['loss'] / len(X_test)),
            'accuracy': float(metrics_accum['accuracy'] / len(X_test)),
            'dice_coefficient': float(metrics_accum['dice_coefficient'] / len(X_test))
        }

        # Calculate additional metrics on a subset to save memory
        subset_size = min(100, len(X_test))
        indices = np.random.choice(len(X_test), subset_size, replace=False)
        X_subset = X_test[indices]
        y_subset = y_test[indices]

        y_pred = self.predict(X_subset)
        y_test_flat = y_subset.squeeze().flatten()
        y_pred_flat = y_pred.flatten()

        # Calculate metrics
        metrics['f1_score'] = float(f1_score(y_test_flat, y_pred_flat, zero_division=1))
        metrics['jaccard_score'] = float(jaccard_score(y_test_flat, y_pred_flat, zero_division=1))

        # Confusion matrix
        cm = confusion_matrix(y_test_flat, y_pred_flat)
        metrics['confusion_matrix'] = cm.tolist()

        self.metrics = metrics
        return metrics

    def predict(self, images, threshold=0.5):
        """
        Predict segmentation masks for images
        """
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        # Add batch dimension if needed
        if len(images.shape) == 3:
            images = np.expand_dims(images, axis=0)

        # Predict in smaller batches if needed
        if len(images) > 8:  # Predict in smaller batches
            predictions = []
            for i in range(0, len(images), 8):
                batch = images[i:i+8]
                pred_batch = self.model.predict(batch, verbose=0)
                predictions.append(pred_batch)
            predictions = np.concatenate(predictions, axis=0)
        else:
            predictions = self.model.predict(images, verbose=0)

        # Apply threshold to get binary masks
        binary_predictions = (predictions > threshold).astype(np.uint8)

        return binary_predictions.squeeze()

    # Keep other methods the same but with memory optimizations
    def plot_training_history(self, save_path=None):
        """Simplified plotting to save memory"""
        if self.history is None:
            raise ValueError("No training history available. Please train the model first.")

        fig, axes = plt.subplots(1, 2, figsize=(12, 4))

        # Plot loss
        axes[0].plot(self.history.history['loss'], label='Training Loss')
        axes[0].plot(self.history.history['val_loss'], label='Validation Loss')
        axes[0].set_title('Model Loss')
        axes[0].set_ylabel('Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].legend()
        axes[0].grid(True)

        # Plot accuracy
        axes[1].plot(self.history.history['accuracy'], label='Training Accuracy')
        axes[1].plot(self.history.history['val_accuracy'], label='Validation Accuracy')
        axes[1].set_title('Model Accuracy')
        axes[1].set_ylabel('Accuracy')
        axes[1].set_xlabel('Epoch')
        axes[1].legend()
        axes[1].grid(True)

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')  # Lower DPI
        plt.close()  # Close plot to save memory

    def visualize_predictions(self, X_test, y_test, num_samples=3, save_path=None):  # Reduced samples
        """Visualize fewer samples to save memory"""
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        # Select random samples
        indices = np.random.choice(len(X_test), min(num_samples, len(X_test)), replace=False)

        fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))

        if num_samples == 1:
            axes = axes.reshape(1, -1)

        for i, idx in enumerate(indices):
            # Get image and masks
            image = X_test[idx]
            true_mask = y_test[idx].squeeze()
            pred_mask = self.predict(image)

            # Plot original image
            axes[i, 0].imshow(image)
            axes[i, 0].set_title('Original Image')
            axes[i, 0].axis('off')

            # Plot true mask
            axes[i, 1].imshow(true_mask, cmap='gray')
            axes[i, 1].set_title('True Mask')
            axes[i, 1].axis('off')

            # Plot predicted mask
            axes[i, 2].imshow(pred_mask, cmap='gray')
            axes[i, 2].set_title('Predicted Mask')
            axes[i, 2].axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()

    def save_model(self, model_path='/content/unet_oil_spill_model.h5'):
        """Save model"""
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        self.model.save(model_path)
        print(f"Model saved to {model_path}")

# Memory-efficient data loading
def load_preprocessed_data(data_dir, split_name, max_samples=None):
    """
    Load preprocessed images and masks from directory with memory optimization
    """
    images_dir = os.path.join(data_dir, split_name, 'images')
    masks_dir = os.path.join(data_dir, split_name, 'masks')

    if not os.path.exists(images_dir) or not os.path.exists(masks_dir):
        raise ValueError(f"Preprocessed data not found for {split_name}")

    # Get sorted lists of files
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
    mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.png')])

    # Limit samples if specified
    if max_samples:
        image_files = image_files[:max_samples]
        mask_files = mask_files[:max_samples]

    images = []
    masks = []

    for img_file, mask_file in tqdm(zip(image_files, mask_files), desc=f"Loading {split_name} data", total=len(image_files)):
        # Load image
        img_path = os.path.join(images_dir, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32) / 255.0  # Normalize

        # Load mask
        mask_path = os.path.join(masks_dir, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        mask = mask.astype(np.float32) / 255.0  # Normalize to 0-1

        # Add channel dimension to mask
        mask = np.expand_dims(mask, axis=-1)

        images.append(image)
        masks.append(mask)

        # Clear memory periodically
        if len(images) % 100 == 0:
            gc.collect()

    return np.array(images), np.array(masks)

def calculate_class_weights(y_train):
    """
    Calculate class weights for imbalanced dataset
    """
    # Use a subset to save memory
    subset_size = min(1000, len(y_train))
    indices = np.random.choice(len(y_train), subset_size, replace=False)
    y_subset = y_train[indices]

    # Flatten masks and convert to binary
    all_masks = np.concatenate([mask.flatten() for mask in y_subset])
    all_masks_binary = (all_masks > 0.5).astype(np.int32)

    # Count class frequencies
    class_counts = np.bincount(all_masks_binary)

    if len(class_counts) == 1:
        if class_counts[0] > 0:
            class_counts = np.array([class_counts[0], 0])
        else:
            class_counts = np.array([0, class_counts[0]])

    total = np.sum(class_counts)

    class_weights = {
        0: float(total / (2 * class_counts[0])) if class_counts[0] > 0 else 1.0,
        1: float(total / (2 * class_counts[1])) if class_counts[1] > 0 else 1.0
    }

    print(f"Class counts: Background={class_counts[0]}, Oil Spill={class_counts[1]}")
    print(f"Class weights: {class_weights}")
    return class_weights

# Main execution function with memory optimization
def main():
    """
    Main function to train and evaluate the U-Net model with memory optimizations
    """
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # Load preprocessed data with limited samples
    data_dir = '/content/drive/MyDrive/preprocessed_dataset'
    max_samples = 500  # Limit samples to save memory

    print("Loading training data...")
    X_train, y_train = load_preprocessed_data(data_dir, 'train', max_samples)
    print(f"Training data shape: {X_train.shape}, {y_train.shape}")

    print("Loading validation data...")
    X_val, y_val = load_preprocessed_data(data_dir, 'val', max_samples//2)
    print(f"Validation data shape: {X_val.shape}, {y_val.shape}")

    print("Loading test data...")
    X_test, y_test = load_preprocessed_data(data_dir, 'test', max_samples//2)
    print(f"Test data shape: {X_test.shape}, {y_test.shape}")

    # Clear memory
    gc.collect()

    # Build smaller U-Net model
    segmenter = UNetOilSpillSegmenter(input_shape=(256, 256, 3))
    model = segmenter.build_unet(filters=32, dropout_rate=0.3, batch_norm=True)  # Smaller model
    model.summary()

    # Compile model
    segmenter.compile_model(learning_rate=1e-4, loss_function='bce_dice')

    # Train the model with smaller batch size
    print("Training U-Net model...")
    history = segmenter.train(
        X_train, y_train, X_val, y_val,
        epochs=10,  # Reduced epochs
        batch_size=8  # Smaller batch size
    )

    # Evaluate the model
    print("Evaluating model...")
    metrics = segmenter.evaluate(X_test, y_test, batch_size=8)
    print("\nEvaluation Metrics:")
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")

    # Plot training history
    segmenter.plot_training_history(save_path='/content/training_history.png')

    # Visualize predictions
    segmenter.visualize_predictions(X_test, y_test, num_samples=3, save_path='/content/predictions_visualization.png')

    # Save the model
    segmenter.save_model('/content/unet_oil_spill_segmentation_model.h5')

    # Download results
    print("Downloading results...")
    files.download('/content/training_history.png')
    files.download('/content/predictions_visualization.png')
    files.download('/content/unet_oil_spill_segmentation_model.h5')

    print("U-Net training and evaluation completed successfully!")

    return segmenter, metrics

# Run the main function
if __name__ == "__main__":
    segmenter, metrics = main()

Mounted at /content/drive
Loading training data...


Loading train data: 100%|██████████| 500/500 [00:40<00:00, 12.34it/s]


Training data shape: (500, 256, 256, 3), (500, 256, 256, 1)
Loading validation data...


Loading val data: 100%|██████████| 203/203 [00:04<00:00, 48.62it/s]


Validation data shape: (203, 256, 256, 3), (203, 256, 256, 1)
Loading test data...


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

U-Net training and evaluation completed successfully!
